In [1]:
import numpy as np
import pandas as pd
#import seaborn as sns
from glob import glob
from PIL import Image
import matplotlib.pylab as plt
from sklearn.model_selection import train_test_split
from imutils import paths

from subprocess import check_output

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.utils import to_categorical

C:\Users\jdu12\Anaconda3\envs\tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_images = glob("C:/Users/jdu12/Desktop/humpback/train/*jpg")
test_images = glob("C:/Users/jdu12/Desktop/humpback/test/*jpg")
df = pd.read_csv("C:/Users/jdu12/Desktop/humpback/train.csv")

#df["Image"] = df["Image"].map( lambda x : "C:/Users/jdu12/Desktop/humpback/input/train/"+x)
#ImageToLabelDict = dict( zip( df["Image"], df["Id"]))

In [3]:
# load a single image to np array
def get_image(img_path):
    img = load_img(img_path, target_size=(299, 299))#.convert('L')
    img = img_to_array(img)
    #img = preprocess_input(img)
    return img

In [4]:
# accquire images' paths
trainPaths = list(paths.list_images('C:/Users/jdu12/Desktop/humpback/train/'))
print(len(trainPaths))
train = np.array([get_image(img_path) for img_path in trainPaths])
print(train.shape)

9850
(9850, 299, 299, 3)


In [5]:
#a = load_img(trainPaths[0], target_size=(299, 299))
#a.show()

In [6]:
def to_one_hot(labels):
    one_hot = []
    seen_id = set()
    id2class = dict()
    counter = 0
    for id in labels:
        if id not in seen_id:
            seen_id.add(id)
            one_hot.append(counter)
            id2class[id] = counter
            counter += 1
        else:
            one_hot.append(id2class[id])
    one_hot = to_categorical(one_hot, num_classes = 4251)
    return one_hot

In [7]:
labels = df['Id']
labels = to_one_hot(labels)
#print(labels[0:10])

In [8]:
# split the data into train and val
# The dataset is super unbalanced, as there are many classes that contains only 1 image
# As a result, train/val data cannot be split before generating more data by augmentation


In [9]:
# define data generator
#use of an image generator for preprocessing and data augmentation
#train = train.reshape( (-1,299,299,1))
input_shape = train[0].shape
x_train = train.astype("float32")
y_train = labels

# define data generator
image_gen = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    rescale=1./255,
    rotation_range=15,
    width_shift_range=.15,
    height_shift_range=.15,
    horizontal_flip=True)

In [10]:
# acquire the model
from keras.applications.inception_resnet_v2 import InceptionResNetV2
model = InceptionResNetV2(weights=None, include_top=True, classes=4251)

In [12]:
batch_size = 32
epochs = 10

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(image_gen.flow(x_train, y_train, batch_size=batch_size),
          steps_per_epoch=  x_train.shape[0]//batch_size,
          epochs=epochs,
          verbose=1)
          #class_weight=class_weight_dic)



Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[32,8,8,2080] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: block8_9_conv/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](block8_9_mixed/concat, block8_9_conv/kernel/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: loss_1/mul/_10297 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_61216_loss_1/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor